In [ ]:
if 'google.colab' in str(get_ipython()):
  !wget https://raw.githubusercontent.com/iliya-malecki/ggol/main/rules.py
  !wget https://raw.githubusercontent.com/iliya-malecki/ggol/main/matrix_automata.py
  !pip install optuna

In [74]:
import numpy as np
import cv2
import optuna
from rules import CallableRuleset, basic_convolution, fast_inv_gaussian_activation
import cv2

def run_for(n, field, rules):
    for _ in range(n):
        field = rules(field)
    return field


def build_kernel(corner, side, center):
    return np.array([
        [corner,  side, corner],
        [side  ,center,   side],
        [corner,  side, corner]
    ])


def objective(trial: optuna.Trial):
    corner = trial.suggest_float('corner', -1, 1)
    side = trial.suggest_float('side', -1, 1)
    center = trial.suggest_float('center', -1, 1)
    kernel = build_kernel(corner, side, center)
    rules = CallableRuleset(
        kernel,
        basic_convolution,
        fast_inv_gaussian_activation,
        lambda x: x
    )
    for _ in range(3):
      field = np.random.uniform(0, 1, size=(800, 800)).astype('float32')
      for n in range(500):
          last = field
          field = rules(field)
          meanfield = np.mean(field)
          if meanfield < 0.01 or meanfield > 0.99 or np.allclose(field, last):
            raise optuna.TrialPruned()
    queue = []
    for _ in range(5):
        field = rules(field)
        queue.append(field)
    queue = [cv2.blur(pic, (3,3)) for pic in queue]
    contrast = np.std(queue[-1])
    return (0.1-meanfield)**2, contrast



study = optuna.study.create_study(directions=['minimize', 'maximize'])
study.optimize(objective, n_trials=200)

trials = study.trials_dataframe()



[I 2023-07-09 17:21:00,128] A new study created in memory with name: no-name-e2ef085e-27ef-4c8b-b0a1-d4b75120ffad
[I 2023-07-09 17:21:00,169] Trial 0 pruned. 
[I 2023-07-09 17:21:00,237] Trial 1 pruned. 
[I 2023-07-09 17:21:00,372] Trial 2 pruned. 
[I 2023-07-09 17:21:01,727] Trial 3 pruned. 
[I 2023-07-09 17:21:01,773] Trial 4 pruned. 
[I 2023-07-09 17:21:01,861] Trial 5 pruned. 
[I 2023-07-09 17:21:25,294] Trial 6 finished with values: [0.04438313738097136, 0.1127995178103447] and parameters: {'corner': 0.9584881154929366, 'side': -0.2153629109819024, 'center': -0.679569675913283}. 
[I 2023-07-09 17:21:25,342] Trial 7 pruned. 
[I 2023-07-09 17:21:49,338] Trial 8 finished with values: [0.231275428688287, 0.15063531696796417] and parameters: {'corner': -0.6171995674176125, 'side': 0.9644903201473634, 'center': 0.7074618962087693}. 
[I 2023-07-09 17:21:49,379] Trial 9 pruned. 
[I 2023-07-09 17:21:49,437] Trial 10 pruned. 
[I 2023-07-09 17:22:12,980] Trial 11 finished with values: [0.209

In [99]:
trials.query('state=="COMPLETE"')

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_center,params_corner,params_side,system_attrs_nsga2:generation,state
6,6,0.044383,0.112800,2023-07-09 17:21:01.864245,2023-07-09 17:21:25.294539,0 days 00:00:23.430294,-0.679570,0.958488,-0.215363,0,COMPLETE
8,8,0.231275,0.150635,2023-07-09 17:21:25.344521,2023-07-09 17:21:49.338340,0 days 00:00:23.993819,0.707462,-0.617200,0.964490,0,COMPLETE
11,11,0.209528,0.084064,2023-07-09 17:21:49.440053,2023-07-09 17:22:12.980254,0 days 00:00:23.540201,-0.206980,-0.736481,0.225719,0,COMPLETE
17,17,0.143302,0.089802,2023-07-09 17:22:13.274326,2023-07-09 17:22:36.397353,0 days 00:00:23.123027,-0.330476,-0.149978,0.759942,0,COMPLETE
24,24,0.175806,0.062771,2023-07-09 17:22:37.028582,2023-07-09 17:22:58.859919,0 days 00:00:21.831337,-0.059565,-0.752789,0.177202,0,COMPLETE
28,28,0.214216,0.161113,2023-07-09 17:22:59.050858,2023-07-09 17:23:23.224785,0 days 00:00:24.173927,0.519633,-0.543246,0.874362,0,COMPLETE
35,35,0.310540,0.142763,2023-07-09 17:23:23.746198,2023-07-09 17:23:48.072302,0 days 00:00:24.326104,-0.904300,-0.878563,0.640768,0,COMPLETE
38,38,0.736124,0.073917,2023-07-09 17:23:48.180368,2023-07-09 17:24:10.377223,0 days 00:00:22.196855,0.723257,-0.723085,-0.211728,0,COMPLETE
40,40,0.780943,0.017714,2023-07-09 17:24:10.430691,2023-07-09 17:24:31.080537,0 days 00:00:20.649846,-0.936622,0.289078,0.764220,0,COMPLETE
57,57,0.061011,0.110362,2023-07-09 17:24:34.602234,2023-07-09 17:24:56.170353,0 days 00:00:21.568119,0.729096,-0.995740,0.351533,0,COMPLETE


In [96]:
m = (
    trials
    .query('state=="COMPLETE"')
    .sort_values('values_0')
    .apply(
        lambda row:
        build_kernel(*row[['params_center',	'params_corner',	'params_side']]), axis=1
    )
    .__iter__()
)

In [92]:
next(m)

array([[ 0.24879229, -0.8920062 ,  0.24879229],
       [-0.8920062 ,  0.46585773, -0.8920062 ],
       [ 0.24879229, -0.8920062 ,  0.24879229]])